In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
try:
    xrange = xrange
except:
    xrange = range

In [3]:
env = gym.make('CartPole-v0')

[2018-01-30 09:49:53,994] Making new env: CartPole-v0


In [4]:
gamma = 0.99

def discount_rewards(r):
    # r is a 1D float array of rewards
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r